# Genres

In [63]:
#!pip install kagglehub
import kagglehub
import os
import shutil
from urllib.request import urlretrieve
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import pickle

from datasketch import MinHash, MinHashLSH


In [64]:
def load_data():
    # Contains movieId, title, genres   
    ml_movies = pd.read_csv('data/ml-32m/movies.csv')
    
    # Contains userId, movieId, rating, timestamp
    ml_ratings = pd.read_csv('data/ml-32m/ratings.csv')
    
    # Contains userId, movieId, tag, timestamp
    ml_tags = pd.read_csv('data/ml-32m/tags.csv')

    # Contains movieId, imdbId, tmdbId
    ml_links = pd.read_csv('data/ml-32m/links.csv')

    # IMDB Dataset:
    imdb = pd.read_csv('data/imdb/movies.csv')

    return ml_movies, ml_ratings, ml_tags, ml_links, imdb

ml_movies, ml_ratings, ml_tags, ml_links, imdb = load_data()

/var/folders/0l/26n95fy50m782tbz32d5_1g40000gn/T/ipykernel_33946/683732697.py:15: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  imdb = pd.read_csv('data/imdb/movies.csv')


In [65]:
# Preprocessing IMDB:

#drop if description is "Add a plot" because we have no description then
imdb_descriptions = imdb[imdb['description'] != "Add a Plot"]

imdb_descriptions['id'] = imdb_descriptions['id'].str[2:]
imdb_descriptions['id'] = pd.to_numeric(imdb_descriptions['id'], errors='coerce')

#keep what we need
imdb_descriptions = imdb_descriptions[['id', 'description']]

#merge them, so we have a li nk between the two datasets
merged_links = ml_links.merge(imdb_descriptions, left_on='imdbId', right_on='id', how='inner').dropna(subset=['id'])

/var/folders/0l/26n95fy50m782tbz32d5_1g40000gn/T/ipykernel_33946/493108502.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  imdb_descriptions['id'] = imdb_descriptions['id'].str[2:]
/var/folders/0l/26n95fy50m782tbz32d5_1g40000gn/T/ipykernel_33946/493108502.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  imdb_descriptions['id'] = pd.to_numeric(imdb_descriptions['id'], errors='coerce')


In [66]:
#merge the movies with the descriptions if that is what we want
ml_movies_description = ml_movies.merge(merged_links, left_on='movieId', right_on='movieId', how='inner')

ml_movies_description

,movieId,title,genres,imdbId,tmdbId,id,description
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,114709,862.0,114709,A cowboy doll is profoundly threatened and jea...
1,2,Jumanji (1995),Adventure|Children|Fantasy,113497,8844.0,113497,When two kids find and play a magical board ga...
2,6,Heat (1995),Action|Crime|Thriller,113277,949.0,113277,A group of high-end professional thieves start...
3,8,Tom and Huck (1995),Adventure|Children,112302,45325.0,112302,Two best friends witness a murder and embark o...
4,9,Sudden Death (1995),Action,114576,9091.0,114576,A former fireman takes on a group of terrorist...
...,...,...,...,...,...,...,...
51855,292541,The Settlers (2023),Drama|Western,10370812,989589.0,10370812,"A mixed-race Chilean, rides south on an expedi..."
51856,292585,Night Crawlers (2009),Comedy|Horror,985060,147230.0,985060,Blood is thicker than water in this tiny Texas...
51857,292605,Our River... Our Sky (2023),Drama|War,10676126,855800.0,10676126,Baghdad. The last week of 2006. All over the c...
51858,292613,Freelance (2023),Action|Comedy,15744298,897087.0,15744298,An ex special forces operator takes a job to p...


## Re-format the 'genres' column

edit the 'genres' column from a string seperated by '|' to onehot encoding for all movies.

In [5]:
# Split genres and create a one-hot encoded dataframe
genre_dummies = ml_movies_description['genres'].str.get_dummies(sep='|')

# Combine the original dataframe with the one-hot encoded genres
df_genres = pd.concat([ml_movies_description, genre_dummies], axis=1)

df_genres

,movieId,title,genres,imdbId,tmdbId,id,description,(no genres listed),Action,Adventure,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,114709,862.0,114709,A cowboy doll is profoundly threatened and jea...,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,2,Jumanji (1995),Adventure|Children|Fantasy,113497,8844.0,113497,When two kids find and play a magical board ga...,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,6,Heat (1995),Action|Crime|Thriller,113277,949.0,113277,A group of high-end professional thieves start...,0,1,0,...,0,0,0,0,0,0,0,1,0,0
3,8,Tom and Huck (1995),Adventure|Children,112302,45325.0,112302,Two best friends witness a murder and embark o...,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,9,Sudden Death (1995),Action,114576,9091.0,114576,A former fireman takes on a group of terrorist...,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51855,292541,The Settlers (2023),Drama|Western,10370812,989589.0,10370812,"A mixed-race Chilean, rides south on an expedi...",0,0,0,...,0,0,0,0,0,0,0,0,0,1
51856,292585,Night Crawlers (2009),Comedy|Horror,985060,147230.0,985060,Blood is thicker than water in this tiny Texas...,0,0,0,...,0,1,0,0,0,0,0,0,0,0
51857,292605,Our River... Our Sky (2023),Drama|War,10676126,855800.0,10676126,Baghdad. The last week of 2006. All over the c...,0,0,0,...,0,0,0,0,0,0,0,0,1,0
51858,292613,Freelance (2023),Action|Comedy,15744298,897087.0,15744298,An ex special forces operator takes a job to p...,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [67]:
# save df_genres
load = True
if load:
    df_genres = pd.read_csv('data/df_genres.csv')
else:
    df_genres.to_csv('data/df_genres.csv', index=False)


In [68]:
genres_list = ['(no genres listed)',
 'Action',
 'Adventure',
 'Animation',
 'Children',
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Fantasy',
 'Film-Noir',
 'Horror',
 'IMAX',
 'Musical',
 'Mystery',
 'Romance',
 'Sci-Fi',
 'Thriller',
 'War',
 'Western']

In [69]:
# select columns in genres_list 
df_genres = df_genres[['movieId', 'title', 'description'] + genres_list]
df_genres

,movieId,title,description,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),A cowboy doll is profoundly threatened and jea...,0,0,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,2,Jumanji (1995),When two kids find and play a magical board ga...,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,Heat (1995),A group of high-end professional thieves start...,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
3,8,Tom and Huck (1995),Two best friends witness a murder and embark o...,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,9,Sudden Death (1995),A former fireman takes on a group of terrorist...,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51855,292541,The Settlers (2023),"A mixed-race Chilean, rides south on an expedi...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
51856,292585,Night Crawlers (2009),Blood is thicker than water in this tiny Texas...,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0
51857,292605,Our River... Our Sky (2023),Baghdad. The last week of 2006. All over the c...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
51858,292613,Freelance (2023),An ex special forces operator takes a job to p...,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [90]:
import numpy as np
from collections import defaultdict
import pandas as pd


class OptimizedMovieLSH:
    def __init__(self, num_hash_functions=10, num_bands=5):
        self.num_hash_functions = num_hash_functions
        self.num_bands = num_bands
        self.hash_functions = None
        self.precomputed_hashes = None
        self.movie_ids = None
        self.hash_tables = None
        
    def generate_hash_functions(self, num_genres):
        """Generate bit sampling positions for each band."""
        # Create a (num_bands, num_hash_functions) array of bit positions
        self.hash_functions = np.array([
            np.random.choice(num_genres, size=self.num_hash_functions, replace=True)
            for _ in range(self.num_bands)
        ])
    
    def _compute_all_hashes(self, genre_matrix):
        """
        Compute all hashes for all movies at once using vectorized operations.
        
        Args:
            genre_matrix: numpy array of shape (num_movies, num_genres)
        Returns:
            numpy array of shape (num_bands, num_movies) containing hash values
        """
        num_movies = genre_matrix.shape[0]
        # Preallocate array for hash values
        hashes = np.zeros((self.num_bands, num_movies), dtype=np.int32)
        
        # For each band, compute hashes for all movies at once
        for band in range(self.num_bands):
            # Select bits for this band (vectorized operation)
            selected_bits = genre_matrix[:, self.hash_functions[band]]
            
            # Convert bits to integers using binary weights
            # This creates a unique hash value from the selected bits
            powers_of_two = 2 ** np.arange(self.num_hash_functions)
            hashes[band] = selected_bits.dot(powers_of_two)
            
        return hashes
    
    def index_movies(self, df, genres_list):
        """
        Index all movies using vectorized operations.
        """
        if self.hash_functions is None:
            self.generate_hash_functions(len(genres_list))
        
        # Convert DataFrame to numpy array for faster operations
        genre_matrix = df[genres_list].values
        self.movie_ids = np.array(df.index)
        
        # Compute all hashes at once
        self.precomputed_hashes = self._compute_all_hashes(genre_matrix)
        
        # Create hash tables using numpy operations
        self.hash_tables = [defaultdict(list) for _ in range(self.num_bands)]
        
        # Vectorized hash table construction
        for band in range(self.num_bands):
            unique_hashes, inverse_indices = np.unique(self.precomputed_hashes[band], 
                                                     return_inverse=True)
            # Create hash tables using numpy operations
            for i, hash_val in enumerate(unique_hashes):
                matching_movies = self.movie_ids[inverse_indices == i]
                self.hash_tables[band][hash_val] = matching_movies.tolist()
    
    def query(self, query_vector, threshold=2):
        """
        Find similar movies using vectorized operations.
        
        Args:
            query_vector: Binary vector of genre features
            threshold: Minimum number of matching bands
        """
        # Compute query hashes using the same method
        query_hashes = np.zeros(self.num_bands, dtype=np.int32)
        
        for band in range(self.num_bands):
            selected_bits = query_vector[self.hash_functions[band]]
            powers_of_two = 2 ** np.arange(self.num_hash_functions)
            query_hashes[band] = selected_bits.dot(powers_of_two)
        
        # Count matches for each movie using vectorized operations
        candidate_counts = defaultdict(int)
        
        # Use numpy operations to find matching movies
        for band, query_hash in enumerate(query_hashes):
            matching_movies = self.hash_tables[band].get(query_hash, [])
            for movie_id in matching_movies:
                candidate_counts[movie_id] += 1
        
        return {movie_id for movie_id, count in candidate_counts.items() 
                if count >= threshold}

    def save_state(self, filename):
        """Save the LSH state efficiently using numpy."""
        np.savez_compressed(
            filename,
            hash_functions=self.hash_functions,
            precomputed_hashes=self.precomputed_hashes,
            movie_ids=self.movie_ids
        )
    
    def load_state(self, filename):
        """Load the LSH state and rebuild hash tables efficiently."""
        data = np.load(filename)
        self.hash_functions = data['hash_functions']
        self.precomputed_hashes = data['precomputed_hashes']
        self.movie_ids = data['movie_ids']
        
        # Rebuild hash tables efficiently
        self.hash_tables = [defaultdict(list) for _ in range(self.num_bands)]
        for band in range(self.num_bands):
            unique_hashes, inverse_indices = np.unique(self.precomputed_hashes[band], 
                                                     return_inverse=True)
            for i, hash_val in enumerate(unique_hashes):
                matching_movies = self.movie_ids[inverse_indices == i]
                self.hash_tables[band][hash_val] = matching_movies.tolist()


# Initialize and index
lsh_optim = OptimizedMovieLSH(num_hash_functions=128, num_bands=614)
lsh_optim.index_movies(df_genres, genres_list)

# Save state efficiently
lsh_optim.save_state('data/'+'movie_lsh_optimized.npz')

def batch_find_similar_movies(query_movie_ids, lsh=lsh_optim, df=df_genres, genres_list=genres_list, 
                            threshold=2, similarity_threshold=0.3, batch_size=1000):
    """
    Find similar movies for multiple queries efficiently.
    
    Args:
        lsh: OptimizedMovieLSH instance
        df: DataFrame with genre data
        query_movie_ids: List of movie IDs to find similar movies for
        batch_size: Number of queries to process at once
    """
    genre_matrix = df[genres_list].values
    results = {}
    
    # Process queries in batches
    for i in range(0, len(query_movie_ids), batch_size):
        batch_ids = query_movie_ids[i:i + batch_size]
        batch_vectors = genre_matrix[np.searchsorted(df.index, batch_ids)]
        
        for idx, query_id in enumerate(batch_ids):
            candidates = lsh.query(batch_vectors[idx], threshold)
            
            # Compute similarities using vectorized operations
            if candidates:
                candidate_vectors = genre_matrix[np.searchsorted(df.index, list(candidates))]
                query_vec = batch_vectors[idx]
                
                # Vectorized Jaccard similarity computation
                intersection = (candidate_vectors & query_vec).sum(axis=1)
                union = (candidate_vectors | query_vec).sum(axis=1)
                similarities = intersection / np.maximum(union, 1)
                
                # Filter and sort results
                mask = similarities >= similarity_threshold
                similar_movies = list(zip(np.array(list(candidates))[mask], 
                                       similarities[mask]))
                results[query_id] = sorted(similar_movies, key=lambda x: x[1], 
                                         reverse=True)
    
    return results

def movie_recommendation_genres(movieId):
    result_dict = batch_find_similar_movies(query_movie_ids=[movieId])
    return result_dict[movieId]


# The below seems to work!


In [91]:


# Process multiple queries efficiently
query_ids = [1, 2, 3, 4, 5]  # List of movie IDs to find similarities for
results = batch_find_similar_movies(
    lsh=lsh_optim,
    df=df_genres,
    genres_list=genres_list,
    query_movie_ids=query_ids,
    threshold=2,
    similarity_threshold=0.3,
    batch_size=100
)
results

{1: [(1, 1.0),
  (32769, 1.0),
  (41, 1.0),
  (79, 1.0),
  (33328, 1.0),
  (623, 1.0),
  (1274, 1.0),
  (1314, 1.0),
  (34116, 1.0),
  (1368, 1.0),
  (1369, 1.0),
  (1524, 1.0),
  (34469, 1.0),
  (34796, 1.0),
  (34863, 1.0),
  (35022, 1.0),
  (35496, 1.0),
  (35619, 1.0),
  (35665, 1.0),
  (3286, 1.0),
  (36133, 1.0),
  (36177, 1.0),
  (36473, 1.0),
  (37121, 1.0),
  (37461, 1.0),
  (37744, 1.0),
  (38491, 1.0),
  (6564, 1.0),
  (7154, 1.0),
  (7312, 1.0),
  (7354, 1.0),
  (7763, 1.0),
  (40968, 1.0),
  (8206, 1.0),
  (8244, 1.0),
  (8446, 1.0),
  (41673, 1.0),
  (41676, 1.0),
  (42296, 1.0),
  (42442, 1.0),
  (43083, 1.0),
  (10327, 1.0),
  (43164, 1.0),
  (10515, 1.0),
  (10608, 1.0),
  (43551, 1.0),
  (43748, 1.0),
  (11071, 1.0),
  (44638, 1.0),
  (45411, 1.0),
  (12716, 1.0),
  (13098, 1.0),
  (46107, 1.0),
  (46537, 1.0),
  (13831, 1.0),
  (48084, 1.0),
  (15544, 1.0),
  (16025, 1.0),
  (16356, 1.0),
  (17194, 1.0),
  (50056, 1.0),
  (17350, 1.0),
  (17453, 1.0),
  (50462, 1.0),

In [92]:
results[1].__len__()

2577

## Begin using the genres

In [ ]:
def movie_recommendation_genres(movie_id, df_genres, genres_list):
    """Calculate the jaccard similarity between the genres of the input movie and all other movies.
    
    Args:
        movie_id (int): The movieId of the input movie.
        df_genres (pd.DataFrame): A DataFrame containing the genres of all movies.
        
    Returns:
        recommendations (list): A list of movieIds sorted by their similarity to the input movie. [(movieId, similarity), ...]
    """
    # Get the genres of the input movie
    input_genres = df_genres[df_genres['movieId'] == movie_id][genres_list]
    
    # Get the genres of all other movies
    other_genres = df_genres[df_genres['movieId'] != movie_id][genres_list]

    similarities = []
    # Calculate the Jaccard similarity between the input movie and all other movies
    enough_high_jaccards = 0   # do early stopping
    for i in range(len(other_genres)):
        jaccard = np.sum(np.minimum(input_genres.values, other_genres.iloc[i].values)) / np.sum(np.maximum(input_genres.values, other_genres.iloc[i].values))
        similarities.append(jaccard)
        if jaccard > 0.7:
            enough_high_jaccards += 1
        if enough_high_jaccards > 10:
            break

    # Combine the movieIds and similarities
    recommendations = list(zip(df_genres[df_genres['movieId'] != movie_id]['movieId'], similarities))

    # Sort the recommendations by similarity
    recommendations = sorted(recommendations, key=lambda x: x[1], reverse=True)

    return recommendations
    

In [58]:
movie_recommendation_genres(1, df_genres, genres_list)

[(2294, 1.0),
 (3114, 1.0),
 (3754, 1.0),
 (4016, 1.0),
 (4886, 1.0),
 (33463, 1.0),
 (45074, 1.0),
 (53121, 1.0),
 (65577, 1.0),
 (91355, 1.0),
 (103755, 1.0),
 (114240, 1.0),
 (114552, 1.0),
 (115875, 1.0),
 (115879, 1.0),
 (117454, 1.0),
 (131248, 1.0),
 (136016, 1.0),
 (136361, 1.0),
 (166461, 1.0),
 (175625, 1.0),
 (177037, 1.0),
 (181601, 1.0),
 (186159, 1.0),
 (186177, 1.0),
 (192981, 1.0),
 (196693, 1.0),
 (197743, 1.0),
 (200614, 1.0),
 (200630, 1.0),
 (204188, 1.0),
 (206959, 1.0),
 (213207, 1.0),
 (215229, 1.0),
 (225173, 1.0),
 (227526, 1.0),
 (228513, 1.0),
 (234969, 1.0),
 (247988, 1.0),
 (260495, 1.0),
 (266324, 1.0),
 (267944, 1.0),
 (268744, 1.0),
 (269122, 1.0),
 (269152, 1.0),
 (270294, 1.0),
 (276403, 1.0),
 (280550, 1.0),
 (281096, 1.0),
 (284739, 1.0),
 (286131, 1.0),
 (289983, 1.0),
 (673, 0.8333333333333334),
 (4306, 0.8333333333333334),
 (8444, 0.8333333333333334),
 (26340, 0.8333333333333334),
 (36397, 0.8333333333333334),
 (47124, 0.8333333333333334),
 (51939

# Text explaination of setup

Additional differences:

Memory Efficiency:

Traditional approach often requires significant memory for shingle sets and minhash signatures
Our approach only needs the original binary matrix and sampled bit positions


Speed:

Traditional pipeline has multiple processing steps
Our approach uses direct bit operations which are very fast on modern CPUs


Accuracy:

Both approaches approximate Jaccard similarity
Our approach might be more accurate for binary data since we're not introducing the additional approximation of minhashing


Use Cases:

Traditional: Better for text documents or other data requiring shingling
Our approach: Better for naturally binary data or pre-computed feature vectors



The main insight is that when you already have binary vectors, you can skip the complexity of shingling and minhashing since your data is already in an ideal format for LSH. Would you like me to elaborate on any of these differences?

### Key Differences Summary:
"""
1. Input Format:
   Traditional: Text → needs shingling → sparse high-dimensional binary matrix
   Our Version: Already have dense binary matrix → use directly

2. Dimensionality Reduction:
   Traditional: Uses MinHash to reduce large shingle sets to small signatures
   Our Version: No reduction needed (already low-dimensional)

3. Similarity Preservation:
   Traditional: MinHash approximates Jaccard similarity of shingle sets
   Our Version: Bit sampling directly preserves Jaccard similarity of binary vectors

4. Hashing Mechanism:
   Traditional: Complex hash functions for MinHash + LSH
   Our Version: Simple bit sampling + binary weights

5. Computational Efficiency:
   Traditional: Three steps (shingling → minhash → LSH)
   Our Version: Single step (bit sampling LSH)

6. Memory Usage:
   Traditional: Needs to store shingles, signatures, and LSH tables
   Our Version: Only stores bit positions and hash tables
"""